# nural profet

In [184]:
import pandas as pd 
import numpy as np 
from pmdarima.model_selection import train_test_split
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_percentage_error
import mlflow
import warnings
warnings.filterwarnings("ignore")

In [121]:
df = pd.read_csv("train_data.csv")
df

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
91295,2017-12-27,1,50,38
91296,2017-12-28,1,50,52
91297,2017-12-29,1,50,59
91298,2017-12-30,1,50,66


## Phase 1 : Preprocess

In [122]:
"""In pre process first grouping by items i.e. 50 
   then by droping unique columns geting final X and Y dataframe"""
def pre_pros(df):
    y = {}
    X = {}
    for item,data in df.groupby("item"):
        data = data.drop([col for col in data.columns if data[col].nunique()==1], axis=1)
        data.rename(columns={"sales" : f"sales_{item}"}, inplace=True)
        X.update({"date" : data["date"].values})
        y.update({f"sales_{item}" : data[f"sales_{item}"].values})
    df_X = pd.DataFrame(X)
    df_y = pd.DataFrame(y)
    df_final = pd.concat([df_X, df_y], axis=1)
    
    return df_final

In [123]:
proc_data = pre_pros(df)

In [124]:
proc_data

,date,sales_1,sales_2,sales_3,sales_4,sales_5,sales_6,sales_7,sales_8,sales_9,...,sales_41,sales_42,sales_43,sales_44,sales_45,sales_46,sales_47,sales_48,sales_49,sales_50
0,2013-01-01,13,33,15,10,11,31,25,33,18,...,6,21,22,20,37,30,17,21,18,30
1,2013-01-02,11,43,30,11,6,36,23,37,23,...,15,24,27,15,40,30,15,26,10,32
2,2013-01-03,14,23,14,8,8,18,34,38,25,...,5,14,19,11,42,30,5,25,17,25
3,2013-01-04,13,18,10,19,9,19,36,54,22,...,9,22,29,22,49,37,13,26,22,32
4,2013-01-05,10,34,23,12,8,31,38,51,29,...,13,18,34,19,52,28,12,28,15,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2017-12-27,14,55,26,16,6,32,45,58,37,...,10,23,29,22,65,43,12,31,17,38
1822,2017-12-28,19,50,21,20,15,51,36,49,34,...,9,19,28,19,54,41,15,37,21,52
1823,2017-12-29,15,50,38,25,17,55,57,54,46,...,22,15,32,36,77,48,21,39,23,59
1824,2017-12-30,27,56,27,19,22,55,60,75,37,...,20,35,58,21,63,56,19,50,29,66


## Phase 2 : NP Build Model

In [125]:
def train_test_data(data, split_test = 0.2):
    train, test = train_test_split(data, test_size = split_test) # splite by dates 
    return {"train" : train, "test" : test}

In [126]:
dict_data = train_test_data(proc_data, 0.1)

In [127]:
dict_data

{'train':             date  sales_1  sales_2  sales_3  sales_4  sales_5  sales_6  \
 0     2013-01-01       13       33       15       10       11       31   
 1     2013-01-02       11       43       30       11        6       36   
 2     2013-01-03       14       23       14        8        8       18   
 3     2013-01-04       13       18       10       19        9       19   
 4     2013-01-05       10       34       23       12        8       31   
 ...          ...      ...      ...      ...      ...      ...      ...   
 1638  2017-06-27       32       53       39       27       16       67   
 1639  2017-06-28       50       63       40       18       14       70   
 1640  2017-06-29       28       64       35       32       23       61   
 1641  2017-06-30       28       82       45       29       22       74   
 1642  2017-07-01       31       83       41       37       26       83   
 
       sales_7  sales_8  sales_9  ...  sales_41  sales_42  sales_43  sales_44  \
 0      

In [128]:
"""building nuralprofet model for each item i.e. 50 models"""
def model_build_NP(data):    
    """using only train data"""
    train = data["train"]
    train["date"] = pd.to_datetime(train["date"], format="%Y-%m-%d") # date time formate
    model_dict = {}
    col_names = train.columns
    for y_indx in range(1, len(train.columns)):
        tsdf = train.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict
        tsdf.columns = ["ds", "y"] # NP formate colums
        # model traning
        npf = NeuralProphet(num_hidden_layers = 2) # n_lags= 7 perform good but issue in 30 days data creation for fourcast
        npf.fit(tsdf, freq='D')
        model_dict[item_name] = npf
    return model_dict

In [129]:
dict_models = model_build_NP(dict_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 167


  0%|          | 0/131 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.99E-02, min: 1.35E+00


  0%|          | 0/131 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.99E-02, min: 1.35E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.32E-02
Epoch[167/167]: 100%|███████████| 167/167 [00:12<00:00, 13.78it/s, SmoothL1Loss=0.0132, MAE=3.46, RMSE=4.36, RegLoss=0]
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 167


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.05E-01, min: 1.15E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.82E-02
Epoch[167/167]: 100%|███████████| 167/167 [00:11<00:00, 14.10it/s, SmoothL1Loss=0.0121, MAE=4.06, RMSE=5.09, RegLoss=0]
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 167


  0%|          | 0/131 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.99E-02, min: 7.14E-01


  0%|          | 0/131 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.99E-02, min: 1.15E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.32E-02
Epoch[167/167]: 100%|██████████| 167/167 [00:11<00:00, 14.22it/s, SmoothL1Loss=0.00741, MAE=6.48, RMSE=8.09, RegLoss=0]


In [130]:
dict_models

{'sales_1': <neuralprophet.forecaster.NeuralProphet at 0x1fc23072070>,
 'sales_2': <neuralprophet.forecaster.NeuralProphet at 0x1fc1bb9a910>,
 'sales_3': <neuralprophet.forecaster.NeuralProphet at 0x1fc1cece340>,
 'sales_4': <neuralprophet.forecaster.NeuralProphet at 0x1fc22d985e0>,
 'sales_5': <neuralprophet.forecaster.NeuralProphet at 0x1fc23056970>,
 'sales_6': <neuralprophet.forecaster.NeuralProphet at 0x1fc2307c220>,
 'sales_7': <neuralprophet.forecaster.NeuralProphet at 0x1fc23057790>,
 'sales_8': <neuralprophet.forecaster.NeuralProphet at 0x1fc2307ba90>,
 'sales_9': <neuralprophet.forecaster.NeuralProphet at 0x1fc23087610>,
 'sales_10': <neuralprophet.forecaster.NeuralProphet at 0x1fc23073ca0>,
 'sales_11': <neuralprophet.forecaster.NeuralProphet at 0x1fc23019b50>,
 'sales_12': <neuralprophet.forecaster.NeuralProphet at 0x1fc23073550>,
 'sales_13': <neuralprophet.forecaster.NeuralProphet at 0x1fc2308aee0>,
 'sales_14': <neuralprophet.forecaster.NeuralProphet at 0x1fc23076760>,
 

## phase 3 : model evaluation 

In [181]:
def eval_model(model, data):
    """using only train data"""
    train = data["train"]
    test = data["test"]
    
    train["date"] = pd.to_datetime(train["date"], format="%Y-%m-%d") # date time formate
    test["date"] = pd.to_datetime(test["date"], format="%Y-%m-%d")
    
    eval_dict = {}
    col_names = train.columns

    for y_indx in range(1, len(train.columns)):
        item_dict_eval = {}
        for name,dat in data.items():
            tsdf = dat.iloc[:, [0, y_indx]]
            item_name = col_names[y_indx] # item key for model_dict

            tsdf.columns = ["ds", "y"] # NP formate colums
            # model pred and eval
            npf = model[item_name]
            pred = npf.predict(tsdf)
            mape = mean_absolute_percentage_error(pred.iloc[:,1], pred.iloc[:,2]) # change with lag eg. lag 7 : pred.iloc[7:,1], pred.iloc[7:,2]
            item_dict_eval[f"MAPE_{name}"] = mape
        eval_dict[f"{item_name}"] = item_dict_eval
    """overall MAPE""" 
    avg_mape_train = []
    avg_mape_test = []
    for i in eval_dict.values():
        avg_mape_train.append(i["MAPE_train"])
        avg_mape_test.append(i["MAPE_test"]) 
    avg_mape_train = sum(avg_mape_train)/50 # 50 no of items
    avg_mape_test = sum(avg_mape_test)/50 # 50 no of items
    eval_dict["AVG_accuracy_Train"] = 1 - avg_mape_train
    eval_dict["AVG_accuracy_Test"] = 1 - avg_mape_test   
    return eval_dict
    

In [182]:
eval_dic = eval_model(dict_models, dict_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.454% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.939%

In [183]:
eval_dic

{'sales_1': {'MAPE_train': 0.2044386629975771,
  'MAPE_test': 0.21278813136332056},
 'sales_2': {'MAPE_train': 0.12648012078101903,
  'MAPE_test': 0.10764987746246348},
 'sales_3': {'MAPE_train': 0.15540576068596187,
  'MAPE_test': 0.15417509333630983},
 'sales_4': {'MAPE_train': 0.21235317011952226,
  'MAPE_test': 0.1991418260344544},
 'sales_5': {'MAPE_train': 0.23243487207543065,
  'MAPE_test': 0.19564298348157533},
 'sales_6': {'MAPE_train': 0.12551144375948023,
  'MAPE_test': 0.11012502448455297},
 'sales_7': {'MAPE_train': 0.12716642445156448,
  'MAPE_test': 0.13051604726733512},
 'sales_8': {'MAPE_train': 0.1092714477573306,
  'MAPE_test': 0.12037133191106908},
 'sales_9': {'MAPE_train': 0.13197304323409184,
  'MAPE_test': 0.1322190517288165},
 'sales_10': {'MAPE_train': 0.1132398285665379,
  'MAPE_test': 0.11210417713910635},
 'sales_11': {'MAPE_train': 0.11127563287285096,
  'MAPE_test': 0.11600979549189752},
 'sales_12': {'MAPE_train': 0.11378003212941305,
  'MAPE_test': 0.10

## phase 4 : final model building with all data (we can add model compare in future here)

In [135]:
def final_model_build_NP(data): 
    """considering whole data for traning final models"""
    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d") # date time formate
    final_model_dict = {}
    col_names = data.columns
    for y_indx in range(1, len(data.columns)):
        tsdf = data.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict
        tsdf.columns = ["ds", "y"] # NP formate colums
        # model traning
        npf = NeuralProphet(num_hidden_layers = 2) # n_lags= 7 
        npf.fit(tsdf, freq='D')
        final_model_dict[item_name] = npf
      
    return final_model_dict

In [136]:
final_models = final_model_build_NP(proc_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 162


  0%|          | 0/132 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.90E-02, min: 1.30E-01


  0%|          | 0/132 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.90E-02, min: 1.52E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.04E-02
Epoch[162/162]: 100%|███████████| 162/162 [00:12<00:00, 12.57it/s, SmoothL1Loss=0.0125, MAE=3.51, RMSE=4.39, RegLoss=0]
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 162


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.30E-01, min: 4.61E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.19E-01
Epoch[162/162]: 100%|███████████| 162/162 [00:12<00:00, 12.66it/s, SmoothL1Loss=0.0116, MAE=4.13, RMSE=5.14, RegLoss=0]
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 162


  0%|          | 0/132 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 9.49E-02, min: 8.69E-01


  0%|          | 0/132 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 9.49E-02, min: 1.19E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.65E-02
Epoch[162/162]: 100%|██████████| 162/162 [00:12<00:00, 12.70it/s, SmoothL1Loss=0.00733, MAE=6.56, RMSE=8.17, RegLoss=0]


In [137]:
final_models

{'sales_1': <neuralprophet.forecaster.NeuralProphet at 0x1fc23b1a040>,
 'sales_2': <neuralprophet.forecaster.NeuralProphet at 0x1fc23060df0>,
 'sales_3': <neuralprophet.forecaster.NeuralProphet at 0x1fc2307cd30>,
 'sales_4': <neuralprophet.forecaster.NeuralProphet at 0x1fc239a8880>,
 'sales_5': <neuralprophet.forecaster.NeuralProphet at 0x1fc23baabb0>,
 'sales_6': <neuralprophet.forecaster.NeuralProphet at 0x1fc23c3d610>,
 'sales_7': <neuralprophet.forecaster.NeuralProphet at 0x1fc23c37430>,
 'sales_8': <neuralprophet.forecaster.NeuralProphet at 0x1fc23b648e0>,
 'sales_9': <neuralprophet.forecaster.NeuralProphet at 0x1fc239d4100>,
 'sales_10': <neuralprophet.forecaster.NeuralProphet at 0x1fc23072c10>,
 'sales_11': <neuralprophet.forecaster.NeuralProphet at 0x1fc23aafc10>,
 'sales_12': <neuralprophet.forecaster.NeuralProphet at 0x1fc23cd3430>,
 'sales_13': <neuralprophet.forecaster.NeuralProphet at 0x1fc23d6e5b0>,
 'sales_14': <neuralprophet.forecaster.NeuralProphet at 0x1fc23dbf610>,
 

## Phase 5 : fourcast next month with final models

In [154]:
def fourcast(final_models,process_data,period=30):    
    process_data["date"] = pd.to_datetime(process_data["date"], format="%Y-%m-%d") # date time formate

    col_names = process_data.columns
    
    dict_fourcast = {}
    for y_indx in range(1, len(process_data.columns)):
        tsdf = process_data.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict

        tsdf.columns = ["ds", "y"] # NP formate colums
        # model pred and eval
        npfinal = final_models[item_name]
        future = npfinal.make_future_dataframe(tsdf, periods= period) 
        forecast = npfinal.predict(future)
        dict_fourcast[item_name] = forecast["yhat1"].values
        df_fourcast = pd.DataFrame(dict_fourcast)
        final_fourcast = pd.concat([future["ds"], df_fourcast], axis=1)
        final_fourcast.rename(columns = {"ds" : "Date"}, inplace = True)
    return final_fourcast
    

In [156]:
final_fourcast = fourcast(final_models, proc_data, 30)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667%

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.945% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667%

In [157]:
final_fourcast

,Date,sales_1,sales_2,sales_3,sales_4,sales_5,sales_6,sales_7,sales_8,sales_9,...,sales_41,sales_42,sales_43,sales_44,sales_45,sales_46,sales_47,sales_48,sales_49,sales_50
0,2018-01-01,11.359156,35.424644,19.723595,11.087411,9.813377,34.900227,31.508875,43.950386,26.628590,...,12.260443,18.648516,28.090199,17.514156,43.481125,33.297436,12.103880,29.832226,16.882732,35.597202
1,2018-01-02,14.140992,42.757236,23.223000,14.020658,12.016328,42.303356,38.691879,53.854515,32.344151,...,14.980112,22.375595,33.052452,21.109150,53.245319,40.139599,14.804081,36.801243,20.091303,43.575478
2,2018-01-03,14.685508,43.985077,23.846638,14.113669,12.313349,42.795845,38.933266,54.112358,32.864571,...,15.293009,23.182060,34.431778,21.100288,53.783348,40.795227,15.097801,36.123810,19.803154,44.549351
3,2018-01-04,15.529398,46.954239,25.637947,16.153685,13.885095,45.038696,43.100616,58.188259,35.799572,...,16.355579,24.850800,36.928223,23.488472,57.706516,44.734329,16.652641,39.713135,22.886192,47.946236
4,2018-01-05,17.173433,50.723633,28.551336,16.595592,14.075049,49.380592,45.762459,63.678478,38.654793,...,18.204102,26.970984,40.365440,24.781799,63.637508,48.606369,17.385603,43.637772,23.698809,51.823925
5,2018-01-06,19.296957,54.839729,30.737381,18.310577,15.702939,53.476315,51.215729,68.302879,42.662216,...,19.998190,29.174887,43.289680,26.964376,68.223007,51.095360,18.648399,46.117191,25.663891,57.100220
6,2018-01-07,20.209824,58.309959,33.668949,19.296457,16.715927,57.424774,53.769585,73.007889,44.963825,...,20.591986,31.502962,46.723049,28.142611,75.066925,56.155502,20.082243,50.525024,28.300669,59.266804
7,2018-01-08,11.838045,36.509872,20.432240,11.236192,10.039147,35.621063,32.417168,44.615028,27.203777,...,12.432948,18.752855,28.367178,18.134357,44.779930,34.287666,12.304552,31.093861,17.069271,36.431068
8,2018-01-09,14.650858,43.896965,23.996050,14.212663,12.284648,43.077145,39.714874,54.610756,33.043041,...,15.174465,22.563232,33.413162,21.747066,54.674507,41.191669,15.020792,38.095375,20.299103,44.534328
9,2018-01-10,15.219429,45.166149,24.672533,14.344262,12.620101,43.611099,40.056774,54.944881,33.676598,...,15.505055,23.446356,34.866310,21.746899,55.321224,41.894405,15.326606,37.436321,20.026390,45.617722


## Phase 6 : retraning model with new data

# profet 

In [5]:
# pip install prophet

In [7]:
import pandas as pd 
import numpy as np 
from pmdarima.model_selection import train_test_split
from neuralprophet import NeuralProphet
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import mlflow
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv("train_data.csv")
df

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
91295,2017-12-27,1,50,38
91296,2017-12-28,1,50,52
91297,2017-12-29,1,50,59
91298,2017-12-30,1,50,66


# Phase 1 : Preprocess

In [9]:
"""In pre process first grouping by items i.e. 50 
   then by droping unique columns geting final X and Y dataframe"""
def pre_pros(df):
    y = {}
    X = {}
    for item,data in df.groupby("item"):
        data = data.drop([col for col in data.columns if data[col].nunique()==1], axis=1)
        data.rename(columns={"sales" : f"sales_{item}"}, inplace=True)
        X.update({"date" : data["date"].values})
        y.update({f"sales_{item}" : data[f"sales_{item}"].values})
    df_X = pd.DataFrame(X)
    df_y = pd.DataFrame(y)
    df_final = pd.concat([df_X, df_y], axis=1)
    
    return df_final

In [10]:
proc_data = pre_pros(df)

In [11]:
proc_data

,date,sales_1,sales_2,sales_3,sales_4,sales_5,sales_6,sales_7,sales_8,sales_9,sales_10,sales_11,sales_12,sales_13,sales_14,sales_15,sales_16,sales_17,sales_18,sales_19,sales_20,sales_21,sales_22,sales_23,sales_24,sales_25,sales_26,sales_27,sales_28,sales_29,sales_30,sales_31,sales_32,sales_33,sales_34,sales_35,sales_36,sales_37,sales_38,sales_39,sales_40,sales_41,sales_42,sales_43,sales_44,sales_45,sales_46,sales_47,sales_48,sales_49,sales_50
0,2013-01-01,13,33,15,10,11,31,25,33,18,37,37,33,37,22,42,14,13,38,15,22,22,59,14,41,59,16,14,49,34,17,30,22,37,23,28,49,16,21,25,19,6,21,22,20,37,30,17,21,18,30
1,2013-01-02,11,43,30,11,6,36,23,37,23,34,43,35,31,35,33,11,18,51,18,16,28,29,17,25,26,26,10,39,30,25,19,22,30,6,26,48,13,28,23,15,15,24,27,15,40,30,15,26,10,32
2,2013-01-03,14,23,14,8,8,18,34,38,25,32,34,41,50,26,45,12,15,42,20,24,23,33,5,32,42,31,12,57,35,25,33,30,40,15,38,45,12,40,29,12,5,14,19,11,42,30,5,25,17,25
3,2013-01-04,13,18,10,19,9,19,36,54,22,45,52,45,45,32,39,15,19,50,16,24,15,45,16,36,35,25,18,46,46,27,24,24,48,13,38,41,23,45,31,13,9,22,29,22,49,37,13,26,22,32
4,2013-01-05,10,34,23,12,8,31,38,51,29,35,45,46,49,31,47,22,16,56,19,36,18,32,23,31,45,27,13,45,43,22,32,28,31,15,45,48,23,38,29,19,13,18,34,19,52,28,12,28,15,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2017-12-27,14,55,26,16,6,32,45,58,37,55,43,47,49,42,62,16,29,54,31,29,30,44,25,45,65,42,18,58,53,28,34,28,63,19,54,62,15,62,32,24,10,23,29,22,65,43,12,31,17,38
1822,2017-12-28,19,50,21,20,15,51,36,49,34,63,64,49,68,51,82,24,13,69,33,29,30,60,26,46,63,52,18,57,61,31,46,37,36,20,50,58,18,57,22,28,9,19,28,19,54,41,15,37,21,52
1823,2017-12-29,15,50,38,25,17,55,57,54,46,56,60,58,73,42,65,11,27,66,30,51,35,69,20,53,58,33,16,60,59,24,49,37,48,25,48,69,21,64,38,22,22,15,32,36,77,48,21,39,23,59
1824,2017-12-30,27,56,27,19,22,55,60,75,37,78,66,52,70,57,77,28,32,67,40,38,33,63,23,44,70,49,22,81,58,33,42,25,62,23,66,68,23,66,35,18,20,35,58,21,63,56,19,50,29,66


# Phase 2 : NP Build Model

In [12]:
def train_test_data(data, split_test = 0.2):
    train, test = train_test_split(data, test_size = split_test) # splite by dates 
    return {"train" : train, "test" : test}

In [13]:
dict_data = train_test_data(proc_data, 0.1)

In [14]:
dict_data

{'train':             date  sales_1  sales_2  sales_3  sales_4  ...  sales_46  sales_47  sales_48  sales_49  sales_50
 0     2013-01-01       13       33       15       10  ...        30        17        21        18        30
 1     2013-01-02       11       43       30       11  ...        30        15        26        10        32
 2     2013-01-03       14       23       14        8  ...        30         5        25        17        25
 3     2013-01-04       13       18       10       19  ...        37        13        26        22        32
 4     2013-01-05       10       34       23       12  ...        28        12        28        15        35
 ...          ...      ...      ...      ...      ...  ...       ...       ...       ...       ...       ...
 1638  2017-06-27       32       53       39       27  ...        73        18        67        30        56
 1639  2017-06-28       50       63       40       18  ...        68        26        50        31        79
 1640  201

In [17]:
"""building nuralprofet model for each item i.e. 50 models"""
def model_build_NP(data):    
    """using only train data"""
    train = data["train"]
    train["date"] = pd.to_datetime(train["date"], format="%Y-%m-%d") # date time formate
    model_dict = {}
    col_names = train.columns
    for y_indx in range(1, len(train.columns)):
        tsdf = train.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict
        tsdf.columns = ["ds", "y"] # NP formate colums
        # model traning
        npf = Prophet() 
        npf.fit(tsdf)
        model_dict[item_name] = npf
    return model_dict

In [18]:
dict_models = model_build_NP(dict_data)

16:48:19 - cmdstanpy - INFO - Chain [1] start processing
16:48:19 - cmdstanpy - INFO - Chain [1] done processing
16:48:19 - cmdstanpy - INFO - Chain [1] start processing
16:48:19 - cmdstanpy - INFO - Chain [1] done processing
16:48:20 - cmdstanpy - INFO - Chain [1] start processing
16:48:20 - cmdstanpy - INFO - Chain [1] done processing
16:48:20 - cmdstanpy - INFO - Chain [1] start processing
16:48:20 - cmdstanpy - INFO - Chain [1] done processing
16:48:20 - cmdstanpy - INFO - Chain [1] start processing
16:48:20 - cmdstanpy - INFO - Chain [1] done processing
16:48:20 - cmdstanpy - INFO - Chain [1] start processing
16:48:21 - cmdstanpy - INFO - Chain [1] done processing
16:48:21 - cmdstanpy - INFO - Chain [1] start processing
16:48:21 - cmdstanpy - INFO - Chain [1] done processing
16:48:21 - cmdstanpy - INFO - Chain [1] start processing
16:48:21 - cmdstanpy - INFO - Chain [1] done processing
16:48:21 - cmdstanpy - INFO - Chain [1] start processing
16:48:22 - cmdstanpy - INFO - Chain [1]

In [19]:
dict_models

{'sales_1': <prophet.forecaster.Prophet at 0x2dd93022640>,
 'sales_2': <prophet.forecaster.Prophet at 0x2dd93022760>,
 'sales_3': <prophet.forecaster.Prophet at 0x2dd93022820>,
 'sales_4': <prophet.forecaster.Prophet at 0x2dd93016fd0>,
 'sales_5': <prophet.forecaster.Prophet at 0x2dd93022970>,
 'sales_6': <prophet.forecaster.Prophet at 0x2dd93016550>,
 'sales_7': <prophet.forecaster.Prophet at 0x2dd930168b0>,
 'sales_8': <prophet.forecaster.Prophet at 0x2dd93145910>,
 'sales_9': <prophet.forecaster.Prophet at 0x2dd93142cd0>,
 'sales_10': <prophet.forecaster.Prophet at 0x2dd93105f40>,
 'sales_11': <prophet.forecaster.Prophet at 0x2dd93535ee0>,
 'sales_12': <prophet.forecaster.Prophet at 0x2dd93105100>,
 'sales_13': <prophet.forecaster.Prophet at 0x2dd9357f520>,
 'sales_14': <prophet.forecaster.Prophet at 0x2dd9357f3a0>,
 'sales_15': <prophet.forecaster.Prophet at 0x2dd9353cfa0>,
 'sales_16': <prophet.forecaster.Prophet at 0x2dd93065730>,
 'sales_17': <prophet.forecaster.Prophet at 0x2dd

# phase 3 : model evaluation 

In [20]:
def eval_model(model, data):
    """using only train data"""
    train = data["train"]
    test = data["test"]
    
    train["date"] = pd.to_datetime(train["date"], format="%Y-%m-%d") # date time formate
    test["date"] = pd.to_datetime(test["date"], format="%Y-%m-%d")
    
    eval_dict = {}
    col_names = train.columns

    for y_indx in range(1, len(train.columns)):
        item_dict_eval = {}
        for name,dat in data.items():
            tsdf = dat.iloc[:, [0, y_indx]]
            item_name = col_names[y_indx] # item key for model_dict

            tsdf.columns = ["ds", "y"] # NP formate colums
            # model pred and eval
            npf = model[item_name]
            pred = npf.predict(tsdf)
            mape = mean_absolute_percentage_error(pred.iloc[:,1], pred.iloc[:,2]) 
            item_dict_eval[f"MAPE_{name}"] = mape
        eval_dict[f"{item_name}"] = item_dict_eval
    """overall MAPE""" 
    avg_mape_train = []
    avg_mape_test = []
    for i in eval_dict.values():
        avg_mape_train.append(i["MAPE_train"])
        avg_mape_test.append(i["MAPE_test"]) 
    avg_mape_train = sum(avg_mape_train)/50 # 50 no of items
    avg_mape_test = sum(avg_mape_test)/50 # 50 no of items
    eval_dict["AVG_accuracy_Train"] = 1 - avg_mape_train
    eval_dict["AVG_accuracy_Test"] = 1 - avg_mape_test   
    return eval_dict
    

In [21]:
eval_dic = eval_model(dict_models, dict_data)

In [22]:
eval_dic

{'sales_1': {'MAPE_train': 0.3118567361314326,
  'MAPE_test': 0.24141286309997112},
 'sales_2': {'MAPE_train': 0.23568240583300012,
  'MAPE_test': 0.1834004765127847},
 'sales_3': {'MAPE_train': 0.26380748299445544,
  'MAPE_test': 0.20185118725151344},
 'sales_4': {'MAPE_train': 0.3107693821364924,
  'MAPE_test': 0.24341478511464226},
 'sales_5': {'MAPE_train': 0.3282212140600581,
  'MAPE_test': 0.2512178095211391},
 'sales_6': {'MAPE_train': 0.23243924032953545,
  'MAPE_test': 0.1843501610319281},
 'sales_7': {'MAPE_train': 0.2387577256054227,
  'MAPE_test': 0.18462448119114316},
 'sales_8': {'MAPE_train': 0.21810793728691366,
  'MAPE_test': 0.16579713885963537},
 'sales_9': {'MAPE_train': 0.23976927682211066,
  'MAPE_test': 0.19146499707858397},
 'sales_10': {'MAPE_train': 0.22219156269783014,
  'MAPE_test': 0.1705076499769635},
 'sales_11': {'MAPE_train': 0.22130581311503197,
  'MAPE_test': 0.17211863776201092},
 'sales_12': {'MAPE_train': 0.2273283636461636,
  'MAPE_test': 0.171136

# phase 4 : final model building with all data (we can add model compare in future here)

In [23]:
def final_model_build_NP(data): 
    """considering whole data for traning final models"""
    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d") # date time formate
    final_model_dict = {}
    col_names = data.columns
    for y_indx in range(1, len(data.columns)):
        tsdf = data.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict
        tsdf.columns = ["ds", "y"] # NP formate colums
        # model traning
        npf = Prophet() 
        npf.fit(tsdf)
        final_model_dict[item_name] = npf
      
    return final_model_dict

In [24]:
final_models = final_model_build_NP(proc_data)

16:50:23 - cmdstanpy - INFO - Chain [1] start processing
16:50:23 - cmdstanpy - INFO - Chain [1] done processing
16:50:23 - cmdstanpy - INFO - Chain [1] start processing
16:50:23 - cmdstanpy - INFO - Chain [1] done processing
16:50:23 - cmdstanpy - INFO - Chain [1] start processing
16:50:23 - cmdstanpy - INFO - Chain [1] done processing
16:50:24 - cmdstanpy - INFO - Chain [1] start processing
16:50:24 - cmdstanpy - INFO - Chain [1] done processing
16:50:24 - cmdstanpy - INFO - Chain [1] start processing
16:50:24 - cmdstanpy - INFO - Chain [1] done processing
16:50:24 - cmdstanpy - INFO - Chain [1] start processing
16:50:24 - cmdstanpy - INFO - Chain [1] done processing
16:50:25 - cmdstanpy - INFO - Chain [1] start processing
16:50:25 - cmdstanpy - INFO - Chain [1] done processing
16:50:25 - cmdstanpy - INFO - Chain [1] start processing
16:50:25 - cmdstanpy - INFO - Chain [1] done processing
16:50:25 - cmdstanpy - INFO - Chain [1] start processing
16:50:25 - cmdstanpy - INFO - Chain [1]

In [25]:
final_models

{'sales_1': <prophet.forecaster.Prophet at 0x2dd907168e0>,
 'sales_2': <prophet.forecaster.Prophet at 0x2dd94040370>,
 'sales_3': <prophet.forecaster.Prophet at 0x2dd92fd1b50>,
 'sales_4': <prophet.forecaster.Prophet at 0x2dd93f20d90>,
 'sales_5': <prophet.forecaster.Prophet at 0x2dd94118340>,
 'sales_6': <prophet.forecaster.Prophet at 0x2dd94118d30>,
 'sales_7': <prophet.forecaster.Prophet at 0x2dd94118850>,
 'sales_8': <prophet.forecaster.Prophet at 0x2dd935e08b0>,
 'sales_9': <prophet.forecaster.Prophet at 0x2dd948aefd0>,
 'sales_10': <prophet.forecaster.Prophet at 0x2dd939e83d0>,
 'sales_11': <prophet.forecaster.Prophet at 0x2dd948b9040>,
 'sales_12': <prophet.forecaster.Prophet at 0x2dd93d1d7c0>,
 'sales_13': <prophet.forecaster.Prophet at 0x2dd939e8430>,
 'sales_14': <prophet.forecaster.Prophet at 0x2dd948a07f0>,
 'sales_15': <prophet.forecaster.Prophet at 0x2dd937fe820>,
 'sales_16': <prophet.forecaster.Prophet at 0x2dd939d7af0>,
 'sales_17': <prophet.forecaster.Prophet at 0x2dd

# Phase 5 : fourcast next month with final models

In [55]:
def fourcast(final_models,process_data,period=30):    
    process_data["date"] = pd.to_datetime(process_data["date"], format="%Y-%m-%d") # date time formate

    col_names = process_data.columns
    
    dict_fourcast = {}
    for y_indx in range(1, len(process_data.columns)):
        tsdf = process_data.iloc[:, [0, y_indx]]
        item_name = col_names[y_indx] # item key for model_dict
        
        tsdf.columns = ["ds", "y"] # NP formate colums
        # model pred and eval
        npfinal = final_models[item_name]
        future = npfinal.make_future_dataframe(periods= period, include_history=False) 
        forecast = npfinal.predict(future)
        dict_fourcast[item_name] = forecast["yhat"].values
        df_fourcast = pd.DataFrame(dict_fourcast)
        final_fourcast = pd.concat([future["ds"], df_fourcast], axis=1)
        final_fourcast.rename(columns = {"ds" : "Date"}, inplace = True)
    return final_fourcast
    

In [56]:
final_fourcast1 = fourcast(final_models, proc_data, 30)

In [58]:
final_fourcast1

,Date,sales_1,sales_2,sales_3,sales_4,sales_5,sales_6,sales_7,sales_8,sales_9,sales_10,sales_11,sales_12,sales_13,sales_14,sales_15,sales_16,sales_17,sales_18,sales_19,sales_20,sales_21,sales_22,sales_23,sales_24,sales_25,sales_26,sales_27,sales_28,sales_29,sales_30,sales_31,sales_32,sales_33,sales_34,sales_35,sales_36,sales_37,sales_38,sales_39,sales_40,sales_41,sales_42,sales_43,sales_44,sales_45,sales_46,sales_47,sales_48,sales_49,sales_50
0,2018-01-01,12.985044,36.710568,22.229259,13.111308,10.751645,35.706397,33.393508,47.521532,28.731201,46.197571,43.661105,42.330743,50.812915,36.707847,55.240342,14.939308,19.912046,51.635435,25.257486,29.257100,24.084673,49.475954,18.280958,37.450522,48.348721,28.122115,13.231951,54.543767,43.550812,24.922787,34.628129,27.352922,44.299444,15.946828,40.034154,46.875610,16.993280,49.844642,26.256823,18.136559,14.149819,21.089687,32.401213,18.867933,48.377187,36.003788,13.663385,31.277886,18.439793,39.342718
1,2018-01-02,15.639651,43.581355,25.773522,16.093750,13.015092,43.082880,40.416297,57.712273,34.454733,54.953465,51.382387,50.331376,60.839411,42.889600,65.714564,18.185186,23.082207,62.409690,29.944071,34.739973,29.393042,58.989095,22.023253,45.738247,58.703638,34.651942,16.413542,65.016801,51.063167,29.815222,41.968553,31.821178,52.538074,19.015618,47.723545,55.607094,20.716238,58.399808,31.649334,21.316893,16.730039,24.927707,37.238748,22.477640,57.899596,42.652918,16.241997,37.846493,21.499608,47.159162
2,2018-01-03,16.282702,44.430075,26.367537,16.107757,13.355709,43.457675,40.532429,57.446577,34.918296,55.531401,52.175500,51.047117,60.763097,43.195627,65.943514,18.251989,24.187588,61.599343,30.664277,34.977715,28.781784,60.127896,22.520820,45.790194,57.840152,34.303622,16.233139,64.899306,51.870551,29.792993,42.230203,31.971180,52.434637,19.073501,47.878503,55.735528,20.637144,59.641035,32.247911,21.917749,16.776495,25.680489,38.220930,22.254179,58.128030,43.255077,16.479984,37.235145,21.167820,48.034584
3,2018-01-04,16.943584,47.228741,28.246913,18.006822,14.904728,45.450960,44.654300,61.479733,38.185144,60.158793,55.324040,54.650229,63.973368,45.820572,71.120889,19.813277,25.186258,66.710217,31.899062,37.292181,32.366250,63.999294,23.710176,49.473871,62.426855,37.400267,17.587486,70.166656,56.003761,32.322470,45.099791,33.947176,56.120851,20.922340,51.625747,60.159305,22.608571,64.161791,33.965388,23.864638,17.636249,27.385687,40.630639,24.513469,61.910220,47.041795,17.711119,40.532925,23.974697,51.136841
4,2018-01-05,18.492588,50.577074,30.958861,18.358678,15.247390,49.769385,47.243917,66.666959,40.501279,64.749780,59.591344,59.120646,70.106567,48.514807,76.848020,21.016863,27.792668,72.909158,34.220709,39.378454,33.912933,68.112679,25.843771,53.979825,66.880980,39.339330,19.555143,74.923697,59.265863,34.641445,48.649242,37.095290,60.768377,22.194057,55.644068,65.453503,24.107287,69.366898,36.526430,24.663733,19.263431,29.425888,43.531861,25.766943,67.431962,50.585150,18.645780,44.006066,24.631133,54.659629
5,2018-01-06,20.421802,54.351079,32.851716,20.217864,16.543732,53.441561,52.288174,71.070867,44.377169,68.376738,64.453688,62.400004,74.703721,53.086690,80.933997,23.432363,29.392149,77.850513,37.504372,43.012133,36.315088,73.748439,28.216418,57.605679,71.662273,43.418944,19.632489,80.915310,64.418630,37.658484,51.674606,39.601054,65.005266,23.884745,59.781196,69.657609,25.666417,73.274224,39.337679,27.292813,20.796395,31.524547,46.460475,27.644829,71.484849,53.346193,19.833498,46.388392,26.484180,59.498182
6,2018-01-07,21.204999,57.679681,35.691408,21.095477,17.632182,56.887229,54.597584,75.539965,46.817519,74.312311,67.571494,66.893459,79.378977,55.329832,87.169432,23.690478,31.289431,82.676114,39.847515,45.957327,38.646313,79.371506,29.506345,62.288602,77.475795,45.902918,21.090079,85.878880,68.319544,40.014585,56.481372,41.895339,68.122697,25.829757,63.892676,73.332737,27.458732,78.280775,41.736647,28.683530,21.067188,33.918394,49.730641,28.792757,78.255477,57.889461,20

# Phase 6 : retraning model with new data

MlflowException: Run '7c5c3713427e45d685723670f886cc69' not found

NameError: name 'pyfunc_model' is not defined